In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from gensim.models import KeyedVectors
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from random import randint
import tensorflow as tf
from collections import Counter
import pickle

In [ ]:
df_train = pd.read_csv('../input/train.csv',sep=',')
df_sub = pd.read_csv('../input/test.csv',sep=',')

In [ ]:
question_train_list = list(df_train['question_text'])
train_label = list(df_train['target'])
question_sub_list = list(df_sub['question_text'])

len_train_list = [len(sentence.split()) for sentence in question_train_list]
len_sub_list = [len(sentence.split()) for sentence in question_sub_list]

In [ ]:
numSentence_train = len(question_train_list)
numSentence_sub = len(question_sub_list)
maxSeqLength_train = max(len_train_list)
maxSeqLength_sub = max(len_sub_list)

In [ ]:
maxSeqLength = min(maxSeqLength_train, maxSeqLength_sub)
numSentence_train, numSentence_sub, maxSeqLength_train, maxSeqLength_sub

In [ ]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    return re.sub(strip_special_chars, "", string.lower())

In [ ]:
clean_train_list = [cleanSentences(sentence).split()[:maxSeqLength] for sentence in question_train_list]
clean_sub_list = [cleanSentences(sentence).split()[:maxSeqLength] for sentence in question_sub_list]

In [ ]:
voc_to_num = {}
num_to_voc = {}
index = 0
for ts in clean_train_list:
    for word in ts:
        if not word in voc_to_num:
            index = index +1
            voc_to_num[word] = index
            num_to_voc[index] = word

for ss in clean_sub_list:
    for word in ts:
        if not word in voc_to_num:
            index = index +1
            voc_to_num[word] = index
            num_to_voc[index] = word
            
voc_to_num['UNK'] = index+1
num_to_voc[index+1] = 'UNK'
voc_to_num['ZEROPAD'] = 0
num_to_voc[0] = 'ZEROPAD'

In [ ]:
len(voc_to_num)

In [ ]:
wv_model =KeyedVectors.load_word2vec_format("../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
embed_size = 300

In [ ]:
wordVectors = np.zeros((len(voc_to_num), embed_size), dtype='float32')
wordVectors[0] = wv_model[wv_model.index2word[0]]
for i in range(1,len(voc_to_num)):
    try:
        wordVectors[i] = wv_model[num_to_voc[i]]
    except KeyError:
        wordVectors[i] = wv_model['UNK']
wordVectors.shape   

In [ ]:
maxSeqLength

In [ ]:
ids_t = np.zeros((numSentence_train, maxSeqLength), dtype='int32')
fileCounter = 0
for tl in clean_train_list:
    indexCounter = 0
    lsen = len(tl)
    for word in tl:
        try:
            ids_t[fileCounter][maxSeqLength-lsen+indexCounter] = voc_to_num[word]
        except KeyError:
            ids_t[fileCounter][maxSeqLength-lsen+indexCounter] = voc_to_num['UNK']
        indexCounter = indexCounter + 1
    fileCounter = fileCounter +1

ids_t.shape

In [ ]:
ids_s = np.zeros((numSentence_sub, maxSeqLength), dtype='int32')
fileCounter = 0
for sl in clean_sub_list:
    indexCounter = 0
    lsen = len(sl)
    for word in sl:
        try:
            ids_s[fileCounter][maxSeqLength-lsen+indexCounter] = voc_to_num[word]
        except KeyError:
            ids_s[fileCounter][maxSeqLength-lsen+indexCounter] = voc_to_num['UNK']
        indexCounter = indexCounter + 1
    fileCounter = fileCounter +1

ids_s.shape

In [ ]:
train_label_arr = np.asarray(train_label, dtype=np.int32)
split_frac = 0.8
split_idx = int(numSentence_train*0.8)
train_x, val_x = ids_t[:split_idx], ids_t[split_idx:]
train_y, val_y = train_label_arr[:split_idx], train_label_arr[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

In [ ]:
print("\t\t\tShapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

In [ ]:
lstm_size = 32
lstm_layers = 1
batch_size = 512
learning_rate = 0.0005

In [ ]:
# Create the graph object
graph = tf.Graph()
tf.reset_default_graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [ ]:
# Size of the embedding vectors (number of units in the embedding layer)
with graph.as_default():
    embed = tf.nn.embedding_lookup(wordVectors, inputs_)

In [ ]:
with graph.as_default():
    
    def get_a_cell():
        lstm = tf.nn.rnn_cell.LSTMCell(lstm_size)
        attn = tf.contrib.rnn.AttentionCellWrapper(lstm, attn_length=20)
        drop = tf.contrib.rnn.DropoutWrapper(attn, output_keep_prob=keep_prob)
        return drop
    # Your basic LSTM cell
    # lstm = tf.nn.rnn_cell.LSTMCell(lstm_size)
    
    # Add dropout to the cell
    # drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell_fw = tf.contrib.rnn.MultiRNNCell([get_a_cell() for _ in range(lstm_layers)])
    cell_bw = tf.contrib.rnn.MultiRNNCell([get_a_cell() for _ in range(lstm_layers)])
    #lstm_fw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    #cell_fw = tf.contrib.rnn.DropoutWrapper(lstm_fw, output_keep_prob=keep_prob)
    
    #lstm_bw = tf.nn.rnn_cell.LSTMCell(lstm_size)
    #cell_bw = tf.contrib.rnn.DropoutWrapper(lstm_bw, output_keep_prob=keep_prob)
    # Getting an initial state of all zeros
    #initial_state_fw = cell_fw.zero_state(tf.shape(inputs_)[0], tf.float32)
    #initial_state_bw = cell_bw.zero_state(tf.shape(inputs_)[0], tf.float32)

In [ ]:
with graph.as_default():
    # outputs, final_state = tf.nn.dynamic_rnn(cell, embed,initial_state=initial_state)
    #(outputs, outputs_state) = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, embed, initial_state_fw = initial_state_fw,initial_state_bw = initial_state_bw)
    outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, embed, dtype=tf.float32)
    output = tf.concat(outputs,2)

In [ ]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(output[:, -1], 1, activation_fn=tf.sigmoid)
    
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    xx, yy = x[:n_batches*batch_size], y[:n_batches*batch_size]
    #xxx, yyy = x[n_batches*batch_size:], y[n_batches*batch_size:]
    for ii in range(0, len(xx), batch_size):
        yield xx[ii:ii+batch_size], yy[ii:ii+batch_size]

In [ ]:
epochs = 2

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    iteration = 1
    for e in range(epochs):
        #state_fw = sess.run(cell.zero_state(batch_size, tf.float32))
        #state_bw = sess.run(cell.zero_state(batch_size, tf.float32))
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5
                   }
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%200==0:
                val_acc = []
                #val_state_fw = sess.run(cell.zero_state(batch_size, tf.float32))
                #val_state_bw = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1
                           }
                    batch_acc = sess.run([accuracy], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, ".ipynb_checkpoints/sentiment.ckpt")

In [ ]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.ipynb_checkpoints'))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1
               }
        batch_acc = sess.run([accuracy], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

In [ ]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.ipynb_checkpoints'))
    #test_state = sess.run(cell.zero_state(25000, tf.float32))
    feed = {inputs_: ids_s[:25000],
                keep_prob: 1}
    test_prediction= sess.run(predictions, feed_dict=feed)    

In [ ]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.ipynb_checkpoints'))
    #test_state = sess.run(cell.zero_state(31370, tf.float32))
    feed = {inputs_: ids_s[25000:],
                keep_prob: 1}
    test_prediction1= sess.run(predictions, feed_dict=feed)   

In [ ]:
test_label = sum(np.rint(test_prediction).astype(np.int32).tolist(),[])
test_label1 = sum(np.rint(test_prediction1).astype(np.int32).tolist(),[])

In [ ]:
test_label_sub = test_label+test_label1

In [ ]:
len(test_label_sub)

In [ ]:
id_list = df_sub['qid']

In [ ]:
len(id_list), test_label_sub[:10]

In [ ]:
sub_dict={}
sub_dict['qid'] = id_list
sub_dict['prediction'] = test_label_sub
sub_df = pd.DataFrame.from_dict(sub_dict)

In [ ]:
sub_df.to_csv('submission.csv',index=False)
sub_df.head()

In [ ]:
print(os.listdir("../working"))